# IR Mini Project 2
Ali Ghanbari - 970216657

---

# Dataset

1. Download dataset if necessery:

In [2]:
!wget https: // cogcomp.seas.upenn.edu/Data/QA/QC/train_5500.label - O train.txt - nc
!wget https: // cogcomp.seas.upenn.edu/Data/QA/QC/TREC_10.label - O test.txt - nc


--2022-12-20 14:03:11--  ftp://https/
           => '.listing'
Resolving https (https)... failed: No such host is known. .
wget: unable to resolve host address 'https'
//: Scheme missing.
--2022-12-20 14:03:16--  http://cogcomp.seas.upenn.edu/Data/QA/QC/train_5500.label
Resolving cogcomp.seas.upenn.edu (cogcomp.seas.upenn.edu)... 158.130.57.77
Connecting to cogcomp.seas.upenn.edu (cogcomp.seas.upenn.edu)|158.130.57.77|:80... connected.
HTTP request sent, awaiting response... 302 Hotspot login required
Location: http://dorm.net/login?dst=http%3A%2F%2Fcogcomp.seas.upenn.edu%2FData%2FQA%2FQC%2Ftrain%5F5500.label [following]
--2022-12-20 14:03:20--  http://dorm.net/login?dst=http%3A%2F%2Fcogcomp.seas.upenn.edu%2FData%2FQA%2FQC%2Ftrain_5500.label
Resolving dorm.net (dorm.net)... 172.16.1.1
Connecting to dorm.net (dorm.net)|172.16.1.1|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10694 (10K) [text/html]
Saving to: 'train_5500.label.1'

     0K ..........           

2. Define text preprocessor:

In [56]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Aligator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Aligator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Aligator\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [57]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk import WordNetLemmatizer

english_stopwords = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemma = WordNetLemmatizer()


def preprocess_text(txt: str) -> str:
    tokens = word_tokenize(txt)
    tokens = [w for w in tokens if w.isalpha()]
    tokens = [w.lower() for w in tokens]
    tokens = [w for w in tokens if w not in english_stopwords]
    # tokens = [stemmer.stem(w) for w in tokens]
    # tokens = [lemma.lemmatize(w, pos = "v") for w in tokens]
    # tokens = [lemma.lemmatize(w, pos = "n") for w in tokens]
    return ' '.join(tokens)


3. Define data parser:

In [105]:
# data(parent class, child class, query, vector)
from typing import Iterable
class QueryRow:
    def __init__(self, parent_class : str, child_class : str, query: str, vector) -> None:
        self.parent_class = parent_class
        self.child_class = child_class
        self.query = query
        self.vector = vector
    
    def __repr__(self) -> str:
        return f'{self.parent_class}:{self.child_class} {self.query} - {self.vector}'
        

def parse_line(line) -> QueryRow:
    spline = line.split()
    labels = spline[0]
    text = spline[1:-1]
    splbl = labels.split(':')
    parent_class = splbl[0]
    child_class = splbl[1]
    query = preprocess_text(' '.join(text))
    return QueryRow(parent_class, child_class, query, [])


4. Load and preprocess data:

In [106]:
train_data = []
test_data = []
with open('train.txt') as f:
    train_data = [parse_line(line) for line in f.readlines() if line]
with open('test.txt') as f:
    test_data = [parse_line(line) for line in f.readlines() if line]
train_data[:12]


[DESC:manner serfdom develop leave russia - [],
 ENTY:cremat films featured character popeye doyle - [],
 DESC:manner find list celebrities real names - [],
 ENTY:animal fowl grabs spotlight chinese year monkey - [],
 ABBR:exp full form - [],
 HUM:ind contemptible scoundrel stole cork lunch - [],
 HUM:gr team baseball louis browns become - [],
 HUM:title oldest profession - [],
 DESC:def liver enzymes - [],
 HUM:ind name bounty hunter old west - [],
 NUM:date ozzy osbourne born - [],
 DESC:reason heavier objects travel downhill faster - []]

5. Vectorieze queries:

In [107]:
from sklearn.feature_extraction.text import CountVectorizer

all_data = train_data + test_data
all_queries = [d.query for d in all_data]

count_vect = CountVectorizer()
queries = count_vect.fit_transform(all_queries)


for i in range(len(all_data)):
    row = all_data[i]
    row.vector = queries[i]

all_data[:3]

[DESC:manner serfdom develop leave russia -   (0, 6507)	1
   (0, 2025)	1
   (0, 4124)	1
   (0, 6315)	1,
 ENTY:cremat films featured character popeye doyle -   (0, 2754)	1
   (0, 2698)	1
   (0, 1267)	1
   (0, 5633)	1
   (0, 2208)	1,
 DESC:manner find list celebrities real names -   (0, 2759)	1
   (0, 4231)	1
   (0, 1215)	1
   (0, 5989)	1
   (0, 4887)	1]

---
# Single Level Classification

### Train-Test Split

In [150]:
from sklearn.model_selection import train_test_split
import numpy as np

def make_2d(arr):
    nsamples, nx, ny = arr.shape
    return arr.reshape((nsamples,nx*ny))

train_x = make_2d(np.array([q.vector.toarray() for q in train_data]))
train_y = [q.parent_class for q in train_data]
test_x = make_2d(np.array([q.vector.toarray() for q in test_data]))
test_y = [q.parent_class for q in test_data]

test_x.shape

(500, 8152)

### Benchmark

In [153]:
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score

# benchmark(name, accuracy macro, accuracy micro, precision macro, precision micro, recall macro, recall micro, time train, time test)


def benchmark_single_class(classifier, name: str) -> tuple[str, float, float, float, float, float, float, float, float]:
    train_start = time.process_time()
    classifier.fit(train_x, train_y)
    train_end = time.process_time()
    train_time = train_end - train_start
    test_start = time.process_time()
    test_pred = classifier.predict(test_x)
    test_end = time.process_time()
    test_time = test_end - test_start
    acc = accuracy_score(test_y, test_pred)
    pre_macro = precision_score(test_y, test_pred, average='macro')
    pre_micro = precision_score(test_y, test_pred, average='micro')
    rec_macro = recall_score(test_y, test_pred, average='macro')
    rec_micro = recall_score(test_y, test_pred, average='micro')
    return (name, acc, acc, pre_macro, pre_micro, rec_macro, rec_micro, train_time, test_time)


1. Naïve Bayes(Bernoulli)

In [154]:
from sklearn.naive_bayes import BernoulliNB

bnb = BernoulliNB()
sc_bnb_results = benchmark_single_class(bnb, 'Naïve Bayes(Bernoulli)')
sc_bnb_results

c:\Users\Aligator\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('Naïve Bayes(Bernoulli)',
 0.596,
 0.596,
 0.5644107684430265,
 0.596,
 0.4878980286859022,
 0.596,
 0.21875,
 0.0)

2. Naïve Bayes(Multinomial):

In [11]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
sc_mnb_results = benchmark_single_class(mnb, 'Naïve Bayes(Multinomial)')

3. KNN(k=3):

In [12]:
from sklearn.neighbors import KNeighborsClassifier

knn3 = KNeighborsClassifier(3)
sc_knn3_results = benchmark_single_class(knn3, 'KNN(k=3)')


4. KNN(k=4):

In [13]:
from sklearn.neighbors import KNeighborsClassifier

knn4 = KNeighborsClassifier(4)
sc_knn4_results = benchmark_single_class(knn4, 'KNN(k=4)')


5. KNN(k=5):

In [14]:
from sklearn.neighbors import KNeighborsClassifier

knn5 = KNeighborsClassifier(5)
sc_knn5_results = benchmark_single_class(knn5, 'KNN(k=5)')


6. SVM(Gaussian kernel):

In [15]:
from sklearn.svm import SVC

svmg = SVC(kernel='rbf')
sc_svmg_results = benchmark_single_class(svmg, 'SVM(Gaussian kernel)')


7. SVM(Linear kernel):

In [16]:
from sklearn.svm import SVC

svml = SVC(kernel='linear')
sc_svml_results = benchmark_single_class(svml, 'SVM(Linear kernel)[libsvm]')


In [155]:
from sklearn.svm import LinearSVC

svml2 = LinearSVC()
sc_svml2_results = benchmark_single_class(svml2, 'SVM(Linear kernel)[liblinear]')

### Evaluation Table

In [20]:
import pandas as pd

sc_df = pd.DataFrame(data=[sc_bnb_results, sc_mnb_results, sc_knn3_results, sc_knn4_results, sc_knn5_results, sc_svmg_results, sc_svml_results, sc_svml2_results],
                     columns=['name', 'accuracy macro', 'accuracy micro', 'precision macro', 'precision micro', 'recall macro', 'recall micro', 'time train', 'time test'])

sc_df


,name,accuracy macro,accuracy micro,precision macro,precision micro,recall macro,recall micro,time train,time test
0,Naïve Bayes(Bernoulli),0.596,0.596,0.564411,0.596,0.487898,0.596,0.359375,0.000000
1,Naïve Bayes(Multinomial),0.720,0.720,0.753000,0.720,0.692718,0.720,0.375000,0.015625
2,KNN(k=3),0.398,0.398,0.701188,0.398,0.398801,0.398,0.000000,3.250000
3,KNN(k=4),0.400,0.400,0.758205,0.400,0.411082,0.400,0.000000,3.234375
4,KNN(k=5),0.374,0.374,0.760255,0.374,0.351494,0.374,0.000000,3.234375
5,SVM(Gaussian kernel),0.732,0.732,0.805937,0.732,0.721637,0.732,45.453125,4.781250
6,SVM(Linear kernel)[libsvm],0.754,0.754,0.794515,0.754,0.743710,0.754,33.578125,1.109375
7,SVM(Linear kernel)[liblinear],0.752,0.752,0.794668,0.752,0.745614,0.752,0.140625,0.000000


---

# 2 Level Classification

In [156]:
from sklearn.svm import LinearSVC

lsvm = LinearSVC()
lsvm.fit(train_x, train_y)
lsvm_pred = lsvm.predict(test_x)

from sklearn.metrics import classification_report

print(classification_report(test_y, lsvm_pred))

              precision    recall  f1-score   support

        ABBR       1.00      0.78      0.88         9
        DESC       0.72      0.91      0.80       138
        ENTY       0.80      0.67      0.73        94
         HUM       0.62      0.80      0.70        65
         LOC       0.69      0.63      0.66        81
         NUM       0.94      0.69      0.80       113

    accuracy                           0.75       500
   macro avg       0.79      0.75      0.76       500
weighted avg       0.77      0.75      0.75       500



. Group data by parent class

In [157]:
sub_data = {
    'ABBR' : [],
    'DESC' : [],
    'ENTY' : [],
    'HUM' : [],
    'LOC' : [],
    'NUM' : [],
}

for row in train_data:
    sub_data[row.parent_class].append(row)

In [160]:
from sklearn.svm import LinearSVC

sub_classifiers = {
    'ABBR' : LinearSVC(),
    'DESC' : LinearSVC(),
    'ENTY' : LinearSVC(),
    'HUM' : LinearSVC(),
    'LOC' : LinearSVC(),
    'NUM' : LinearSVC(),
}

for parent_class, classifier in sub_classifiers.items():
    rows = sub_data[parent_class]
    data = make_2d(np.array([q.vector.toarray() for q in rows]))
    labels = [q.child_class for q in rows]
    classifier.fit(data, labels)

---

# Clustering using K-means